### Reinforcement learning & LLMs:
#### Usando REINFORCE para aprender los parametros de gemma-2b que maximizan la recompensa acummulada al resolver problemas matemáticos


In [1]:
!pip install -q -U transformers


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
!pip install -q jsonlines


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
!pip install -q math-verify


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [4]:
!pip install -q datasets


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [9]:
!pip install --upgrade typing-extensions
!pip install -q sentence_transformers


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import torch.nn as nn 
from math_verify import parse, verify
import jsonlines
from datasets import Dataset,DatasetDict
import os 
from sentence_transformers import SentenceTransformer
import re

In [2]:
#cargar modelo de embedding [ara calcular recompensas:
from sentence_transformers import SentenceTransformer
model_embedding = SentenceTransformer('all-MiniLM-L6-v2')
reference_text = "To find the next term, list the sequence. Calculate differences. Analyze them. Compute second differences. Analyze pattern. Predict next difference. Calculate next term. Answer."
reference_embedding = model_embedding.encode(reference_text, convert_to_tensor=True)

#### El modelo que usaremos como policy network es gemma-2b

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
tokenizer = AutoTokenizer.from_pretrained("google/gemma-1.1-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-1.1-2b-it", torch_dtype=torch.bfloat16)
model.to(device)

Using device: cuda


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): GemmaRMSNorm((2048,), eps=1e-06)
      )
    )
    (norm): GemmaRMSNorm((2048,),

In [4]:

def custom_generator(filepath):
  try:
    with open(filepath,'r',encoding='utf-8') as f:
      while True:
        question_line = f.readline()
        answer_line = f.readline()
        if not question_line or not answer_line:
          break
        question = question_line.strip()
        answer = answer_line.strip()

        if question and answer:

          yield {'question':question,'answer':answer}

        elif question or answer:

          print(f"Warning: Skipping potentially incomplete pair near question: '{question[:50]}...'")

  except Exception as e:
      print(f"An error occurred: {e}")


file_path = "algebra__sequence_next_term.txt"
math_dataset = Dataset.from_generator(
    custom_generator,
    gen_kwargs={"filepath":file_path}
)
print(f"Loaded {len(math_dataset)} total examples.")
math_dataset.to_json("train.jsonl")
dataset_path = "train.jsonl"
problems = []
with jsonlines.open(dataset_path) as reader:
    for obj in reader:
        problems.append({"question":obj["question"],"answer":obj["answer"]})

Loaded 407060 total examples.


Creating json from Arrow format:   0%|          | 0/408 [00:00<?, ?ba/s]

In [5]:
print(problems[0])

{'question': 'What is the next term in -890, -28, 782, 1534, 2222?', 'answer': '2840'}


#### Policy es la clase que usamos para representar a nuestro lm como la red que maximiza la recompensa, al cambiar sus parametros

#### * para seleccionar la acción , hacemos un sampling desde la distribución de tokens
#### * calculamos log pobability de ese token, esto nos permite tener un rango más grande y reduce costos computacionales

In [6]:
class Policy:
    #gamma es el hiperparametro que usamos psra descontar recompensas futuras 
    def __init__(self,pretrained_model,gamma = 0.99):
        super(Policy,self).__init__()
        self.model = pretrained_model
        self.tokenizer = tokenizer
        self.gamma = gamma
        # guardamos los log probabilities de cada accion
        self.policy_history = []
        #guardamos las recompensas de cada episodio
        self.reward_episode = []
    def forward(self,input_ids):
        output = self.model(input_ids)
        logits = outputs.logits[:,-1,:]
        probs = nn.Softmax(dim=-1)(logits)
        #retorna un tensor del tamano vocab size, con las probabilidades, pasado por un sofmax
        return probs
    def select_action(policy,input_ids):
        probs = policy(input_ids)
        #convertimos a los tokens en una distribucion y hacemos sampling
        dist = torch.distributions.Categorical(probs=probs)
        action = dist.sample()
        #calculamos log_probs
        log_prob = dist.log_prob(action)
        policy.policy_history.append(log_prob)
        return action

### Ajustando los parámetros del modelo
#### * damos prioridad a recompensas mas cercanas 
#### * para clacular loss: 
#### - asignamos recompensas a cada token, todos son cero excepto el ultimo (la respuesta)
#### - ajustamos las recompensas a discounted_rewards[] usando gamma. esto da a cada token generado "credito" por la respuesta
#### - loss se calcula como -= log_prob *  recompensa

In [7]:
def update_policy(policy,optimizer):
    discounted_rewards = []
    running_reward = 0
    #recorrer la lista desde el final
    for r in policy.reward_episode[::-1]:
        running_reward = r + policy.gamma * running_reward
        discounted_rewards.insert(0,running_reward)
    discounted_rewards = torch.tensor(discounted_rewards,dtype=torch.float32).to(device)
    if discounted_rewards.std() > 0:
        discounted_rewards = (discounted_rewards - discounted_rewards.mean()) / (discounted_rewards.std() + 1e-8)
    loss = 0
    #multiplicamos la probabilidad del token por su recompensa
    for log_prob,reward in zip(policy.policy_history,discounted_rewards):
        loss -= log_prob * reward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    policy.policy_history = []
    policy.reward_episode = []

### Por que se usa el negativo en loss?
#### Log probabilities son negativas, la multiplicacion da una respuesta negativa y necesitamos aumentar la probabilidad de buenas acciones

#### tratamos al modelo como un policy network, que genera acciones(tokens)
#### Estado: el prompt
#### Accion: cada unpo de los tokens generados 
#### Recompensa: 1 si la respuesta es correcta, 0.5 por un buen razonamiento, 0 de otro modo



In [8]:
class Policy(nn.Module):
    #gamma es el hiperparametro que usamos psra descontar recompensas futuras 
    def __init__(self,pretrained_model,gamma = 0.99):
        super(Policy,self).__init__()
        self.model = pretrained_model
        self.tokenizer = tokenizer
        self.gamma = gamma
        # guardamos los log probabilities de cada accion
        self.policy_history = []
        #guardamos las recompensas de cada episodio
        self.reward_episode = []
    def forward(self,input_ids):
        outputs = self.model(input_ids)
        logits = outputs.logits[:,-1,:]
        probs = nn.Softmax(dim=-1)(logits)
        #retorna un tensor del tamano vocab size, con las probabilidades, pasado por un sofmax
        return probs
    def select_action(self,input_ids):
        probs = self.forward(input_ids)
        probs = probs.float()
        #convertimos a los tokens en una distribucion y hacemos sampling
        dist = torch.distributions.Categorical(probs=probs)
        action = dist.sample()
        #calculamos log_probs
        log_prob = dist.log_prob(action)
        policy.policy_history.append(log_prob)
        return action
policy = Policy(model)

### Entrenamiento:
#### usamos Adam para actualizar parametros de gemma-2b


In [9]:
def extract_numbers(text):
    return[float(x) for x in re.findall(r'-?\d+\.?\d*', text)]
def calculate_reward(generated_text, gold_answer, prompt):
    reward = 0.0
    numbers = extract_numbers(generated_text)
    sequence = extract_numbers(prompt)
    gold_num = float(gold_answer)

    if len(sequence) >= 2:
        first_diffs = [sequence[i+1] - sequence[i] for i in range(len(sequence)-1)]
        if any(diff in numbers for diff in first_diffs):
            reward += 0.2
        
        if len(first_diffs) >= 2:
            second_diffs = [first_diffs[i+1] - first_diffs[i] for i in range(len(first_diffs)-1)]
            if any(diff in numbers for diff in second_diffs):
                reward += 0.2
            
            if len(second_diffs) >= 2:
                third_diffs = [second_diffs[i+1] - second_diffs[i] for i in range(len(second_diffs)-1)]
                next_second_diff = second_diffs[-1] + third_diffs[-1] if third_diffs else second_diffs[-1]
                next_first_diff = first_diffs[-1] + next_second_diff
                if next_first_diff in numbers:
                    reward += 0.2
            elif len(second_diffs) == 1:
                next_first_diff = first_diffs[-1] + second_diffs[-1]
                if next_first_diff in numbers:
                    reward += 0.2

    if gold_num in numbers:
        reward += 1.0

    generated_embedding = model_embedding.encode(generated_text, convert_to_tensor=True)
    similarity = torch.cosine_similarity(reference_embedding, generated_embedding, dim=0).item()
    reward += similarity * 0.5

    return reward

In [10]:
def train_reasoning(policy, episodes=200, max_length=100):  # Reduced max_length
    optimizer = torch.optim.Adam(policy.parameters(), lr=0.0001)
    for episode in range(episodes):
        problem = problems[episode % len(problems)]
        prompt = f"find the next term in {problem['question'].split(':')[0].strip()}.Think step by step"
        gold_answer = problem["answer"]
        
        input_ids = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=100).input_ids.to(device)
        generated = input_ids.clone()
        
        for _ in range(max_length):
            action = policy.select_action(generated)
            generated = torch.cat([generated, action.unsqueeze(0)], dim=1)
            policy.reward_episode.append(0)
        
        output_text = tokenizer.decode(generated[0], skip_special_tokens=True)
        reward = calculate_reward(output_text, gold_answer,prompt)
        policy.reward_episode[-1] = reward
        
        print(f"Episode {episode}: Question: {prompt}")
        print(f"Generated: {output_text}, Gold: {gold_answer}, Reward: {reward}")
        
        update_policy(policy, optimizer)
        

train_reasoning(policy,episodes=10)

Episode 0: Question: find the next term in What is the next term in -890, -28, 782, 1534, 2222?.Think step by step
Generated: find the next term in What is the next term in -890, -28, 782, 1534, 2222?.Think step by step.

-890, -28, 782, 1534, 2222

The pattern is adding 382 to each previous term.

So the next term will be 3204.The next term in the sequence is 3204.The pattern is adding 382 to each previous term.Therefore, the answer is 3204.The sequence follows the, Gold: 2840, Reward: 0.3390312194824219
Episode 1: Question: find the next term in What comes next.Think step by step
Generated: find the next term in What comes next.Think step by step convertitability fufing ftafiably suspicaling squafiably emphatting fufantly strickting emphatting intermintably interminantly embrating suspictably tewting ?...shaping emphatantly inconcealing suspicantly embraantly depicting intermintably increatically embrating suspictively embrating ?..."" tranf" embrating suspictively scratting emphatam

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacty of 47.53 GiB of which 8.25 MiB is free. Process 2937886 has 47.51 GiB memory in use. Of the allocated memory 46.92 GiB is allocated by PyTorch, and 279.89 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF